In [1]:
from tqdm import tqdm
from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage
import skimage.io as io
from skimage.io import imread, imsave

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("C:/Users/desktop-pc/Desktop/isic2019.csv")
df.head()

,Unnamed: 0,image,age_approx,anatom_site_general,lesion_id,sex,outcome,outcome_idx,path
0,0,ISIC_0000000,55.0,anterior torso,NaN,female,NV,4,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
1,1,ISIC_0000001,30.0,anterior torso,NaN,female,NV,4,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
2,2,ISIC_0000002,60.0,upper extremity,NaN,female,MEL,3,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
3,3,ISIC_0000003,30.0,upper extremity,NaN,male,NV,4,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
4,4,ISIC_0000004,80.0,posterior torso,NaN,male,MEL,3,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...


In [3]:
df.outcome_idx.value_counts()

4    12875
3     4522
1     3323
2     2624
0      867
7      628
5      253
6      239
Name: outcome_idx, dtype: int64

For undersampling the dominant class

In [4]:
'''from sklearn.utils import resample
df_NV = df[df.outcome_idx==4]
df_rest = df[df.outcome_idx!=4]
df_NV_downsampled = resample(df_NV, 
                                 replace=False,    # sample without replacement
                                 n_samples=5000,     # to match minority class
                                 random_state=1) # reproducible results
df = pd.concat([df_NV_downsampled, df_rest])
df.outcome_idx.value_counts() '''

'from sklearn.utils import resample\ndf_NV = df[df.outcome_idx==4]\ndf_rest = df[df.outcome_idx!=4]\ndf_NV_downsampled = resample(df_NV, \n                                 replace=False,    # sample without replacement\n                                 n_samples=5000,     # to match minority class\n                                 random_state=1) # reproducible results\ndf = pd.concat([df_NV_downsampled, df_rest])\ndf.outcome_idx.value_counts() '

In [5]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)

train_df = train_df.reset_index()
test_df = test_df.reset_index()
test_df.to_csv("C:/Users/desktop-pc/Desktop/test.csv")
train_df.head()

,index,Unnamed: 0,image,age_approx,anatom_site_general,lesion_id,sex,outcome,outcome_idx,path
0,19623,19623,ISIC_0064222,75.0,anterior torso,BCN_0000835,male,AK,0,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
1,12220,12220,ISIC_0033623,40.0,NaN,HAM_0004202,female,NV,4,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
2,9223,9223,ISIC_0030626,40.0,NaN,HAM_0001031,male,NV,4,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
3,23382,23382,ISIC_0070146,85.0,upper extremity,BCN_0003657,male,BCC,1,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...
4,5095,5095,ISIC_0026498,35.0,posterior torso,HAM_0000054,female,NV,4,C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_20...


In [6]:
src_dir = "C:/Users/desktop-pc/Downloads/ISIC2019/ISIC_2019_Training_Input/"
dst_dir = "C:/Users/desktop-pc/Desktop/OverSampling/"
train_img = []
final_target_train = []
for image, outcome in zip(train_df.image, train_df.outcome_idx):
    #shutil.copy(src_dir + str(image) + ".jpg", dst_dir + str(image) + ".jpg")
    img = imread(src_dir + str(image) + ".jpg")
    flipud = np.flipud(img)                                 # vertical flip all images
    plt.imsave(dst_dir + str(image) + "flipedud.jpg", flipud)
    train_df = train_df.append({'outcome_idx': outcome, 'path':  dst_dir + str(image) + "flipedud.jpg" }, ignore_index=True) 
    if outcome==0 or outcome==7 or outcome==5 or outcome==6:      
        img_rotate = rotate(img, angle=180, mode = 'wrap')       # rotate
        plt.imsave(dst_dir + str(image) + "rotated.jpg", img_rotate)
        train_df = train_df.append({'outcome_idx': outcome, 'path': dst_dir + str(image) + "rotated.jpg" }, ignore_index=True) 
        fliplr = np.fliplr(img)                                 # horizontal flip
        plt.imsave(dst_dir + str(image) + "flipedlr.jpg", fliplr)
        train_df = train_df.append({'outcome_idx': outcome, 'path': dst_dir + str(image) + "flipedlr.jpg" }, ignore_index=True) 

In [7]:
train_df.shape

(43654, 10)

In [8]:
from sklearn.utils import shuffle
train_df=shuffle(train_df)
train_df.to_csv("C:/Users/desktop-pc/Desktop/train.csv")

In [9]:
train_df.outcome_idx.value_counts()

4    20700
3     7172
1     5336
2     4194
0     2716
7     1992
5      792
6      752
Name: outcome_idx, dtype: int64